In [ ]:
!pip install transformers tokenizer sentencepiece accelerate bitsandbytes einops safetensors

In [ ]:
import pandas as pd

df = pd.read_json("dna_ja_1k.json")

df.head()

In [ ]:
OUTPUT_FILENAME = "elyza_llama_7b_instruct"
MODEL_NAME = "elyza/ELYZA-japanese-Llama-2-7b-instruct" # 評価したいモデルによって変更

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
                                          use_fast=False)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                            # load_in_8bit=True,
                                            torch_dtype=torch.float16,
                                            device_map="auto",
                                            )

In [ ]:

generated_texts = []  # 結果を保存するリスト

for n in range(len(df)):
# for n in range(5): # test

    if n%100 == 0:
        print(n)

    # elyza_llama prompt
    prompt = f"""<s>[INST] <<SYS>>
    あなたは誠実で優秀な日本人のアシスタントです。
    <</SYS>>

    {df.loc[n, "question"]} [/INST]"""

    # 推論の実行
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=1024,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])

    generated_texts.append(output)

In [ ]:
df_result = pd.concat([df, pd.DataFrame(generated_texts, columns=[f"{OUTPUT_FILENAME}_response"])], axis=1)
df_result.head(3)

In [ ]:
df_result[["id", "question", f"{OUTPUT_FILENAME}_response"]].to_csv(f"{OUTPUT_FILENAME}_response.csv", index=False)